In [9]:
import numpy as np

In [10]:
#will be used for the gomory cuts
def dual_simplex_without_print(A,b,c,Basis):
    indexes=list(range(0,len(c)))
    iter=0
    optimal=False
    while True:
        c_B = np.transpose(np.transpose(c)[:,Basis])
        A_B = A[:, Basis]
        A_B_inv=np.linalg.inv(A_B)
        X_B=np.matmul(np.linalg.inv(A_B),b)
        table=np.matmul(A_B_inv,A)
        c_reduced=np.transpose(c)-np.matmul(np.transpose(c_B),table)
        NB=list(set(indexes)-set(Basis))
        z=np.matmul(np.transpose(c_B),X_B)
        
        print("Basis:",Basis)
        print("Reduced Costs:", c_reduced)
        print("RHS or X_B", X_B)
        print("A_B inverse*A (inside table):",table)
        print("Z:",z)

        leave_found=False
        for j in range(len(b)): 
            if X_B[j]<0 and leave_found==False:
                leaving_index=Basis[j]
                leaving_ind=j
                leave_found=True #terminates loop and ensures Bland's Rule is applied
        if leave_found==False:
            print("OPTIMAL TABLE VIA DUAL SIMPLEX.")
            optimal=True
            break

        row=table[leaving_ind:leaving_ind+1]
        
        min_ratios=[1000000] #is an initializer
        for item in NB:
            if row[:,item]<0:
                min_ratio=c_reduced[:,item]/row[:,item]
                min_ratios.append(min_ratio)
                if min_ratio<min_ratios[min_ratios.index(min_ratio)-1]: #<, not =< ensures Bland's
                    entering_index=item
        
        if len(min_ratios)==1:
            print("PROBLEM INFEASIBLE VIA DUAL SIMPLEX.")
            break

        Basis.remove(leaving_index)
        Basis.append(entering_index)
        iter+=1
    return c_B, A_B, A_B_inv, X_B, table, c_reduced,z,NB,Basis,optimal

In [11]:
def gomory(A,b,c,Basis):
    c_B, A_B, A_B_inv, X_B, table, c_reduced, z, NB, Basis, optimal = dual_simplex_without_print(A,b,c,Basis)
    #getting the optimal table via dual simplex
    
    leave_found=False
    gom_optimal=False #gomory optimal, ie dual simplex optimal and all basic variables are integers
    iter=0
    
    while gom_optimal==False and optimal==True: 
        #optimal comes from dual simplex; used here to ensure that dual simplex does not give infeasibility
        #if dual simplex gives infeasibility, the algortihm is terminated
        #if gomory optimal is found, the algorithm is terminated
        
        print("ITERATION:",iter)
        c_B, A_B, A_B_inv, X_B, table, c_reduced, z, NB, Basis, optimal = dual_simplex_without_print(A,b,c,Basis)
        
        print("Basis:",Basis)
        print("Reduced Costs:", c_reduced)
        print("RHS or X_B", X_B)
        print("A_B inverse*A (inside table):",table)
        print("Z:",z)
        
        if optimal==True and (all([float(i).is_integer() for i in (X_B)]))==True:
            print("OPTIMAL SOLUTION FOR IP")
            gom_optimal=True
            
        else:
            for item in X_B:
                if item%1 !=0 and leave_found==False: #leave_found ensures Bland's Rule
                    print("A BASIC VARIABLE IS NOT AN INTEGER")
                    float_index=list(X_B).index(item) 
                    leave_found==True 
                    leaving_var_index=Basis[float_index] 
                    float_row=table[float_index:float_index+1]

            cut=[] #LHS of the gomory cut
            for i in range(float_row.shape[1]):
                value=float(float_row[:,i])
                if value==0:
                    cut.append(0)
                elif value>0:
                    cut.append((value%1)*(-1))
                else:
                    cut.append(1+(value%(-1)))
                    
            table=np.vstack((table,np.array(cut))) #the cut is added as a row to the table via vstack
            table=np.array(table)
            table=np.asmatrix(table)
            A=np.vstack((np.array(A),np.array(cut))) #the cut is added as a constraint to the A matrix via vstack
            A=np.asmatrix(A)

            zeros=[]
            for i in range(A.shape[0]):
                zeros.append(0)
            zeros=np.array(zeros).reshape(1,A.shape[0])
            A=np.column_stack((np.array(A),np.transpose(zeros)))
            A=np.asmatrix(A)
            #for the new variable, 0s are added as a column to the A matrix
            b=np.vstack((np.array(b),np.array(float(X_B[float_index])%1*(-1))))
            b=np.asmatrix(b)
            #the LHS for the cut is added to the b matrix
            c=np.vstack((np.array(c),np.array([[0]])))
            c=np.asmatrix(c)
            #For the new variable, 0 is added to the c matrix
        
            print("CUT LHS:",cut)
            print("CUT RHS:",np.array(float(X_B[float_index])%1*(-1)))
            print("NEW TABLE(LHS PART):",table)
            
            row=table[-1:] #the leaving variable is the newly added one
                            #so, the min ratio test is done via the last row on the table
            min_ratios=[1000000] #is an initializer
            for item in NB:
                if row[:,item]<0:
                    min_ratio=c_reduced[:,item]/row[:,item]
                    min_ratios.append(min_ratio)
                    if min_ratio<min_ratios[min_ratios.index(min_ratio)-1]: #<, not =< ensures Bland's
                        entering_index=item #index from entire basis

            Basis.append(entering_index) #winner of the min-ratio test enters basis
            print("Index of the variable entering Basis:",entering_index)
            iter+=1

In [12]:
A=np.matrix([[-2,-4,-3,-1,1]])
b=np.transpose(np.matrix([-23]))
c=np.transpose(np.matrix([10,24,10,2,0]))
Basis=[4]
gomory(A,b,c,Basis)

Basis: [4]
Reduced Costs: [[10. 24. 10.  2.  0.]]
RHS or X_B [[-23.]]
A_B inverse*A (inside table): [[-2. -4. -3. -1.  1.]]
Z: [[0.]]
Basis: [1]
Reduced Costs: [[-2.  0. -8. -4.  6.]]
RHS or X_B [[5.75]]
A_B inverse*A (inside table): [[ 0.5   1.    0.75  0.25 -0.25]]
Z: [[138.]]
OPTIMAL TABLE VIA DUAL SIMPLEX.
ITERATION: 0
Basis: [1]
Reduced Costs: [[-2.  0. -8. -4.  6.]]
RHS or X_B [[5.75]]
A_B inverse*A (inside table): [[ 0.5   1.    0.75  0.25 -0.25]]
Z: [[138.]]
OPTIMAL TABLE VIA DUAL SIMPLEX.
Basis: [1]
Reduced Costs: [[-2.  0. -8. -4.  6.]]
RHS or X_B [[5.75]]
A_B inverse*A (inside table): [[ 0.5   1.    0.75  0.25 -0.25]]
Z: [[138.]]
A BASIC VARIABLE IS NOT AN INTEGER
CUT LHS: [-0.5, -0.0, -0.75, -0.25, 0.75]
CUT RHS: -0.75
NEW TABLE(LHS PART): [[ 0.5   1.    0.75  0.25 -0.25]
 [-0.5  -0.   -0.75 -0.25  0.75]]
Index of the variable entering Basis: 0
ITERATION: 1
Basis: [1, 0]
Reduced Costs: [[ 0.  0. -5. -3.  3.  0.]]
RHS or X_B [[5. ]
 [1.5]]
A_B inverse*A (inside table): [[ 0.